In [ ]:
import os
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage

load_dotenv()

GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    google_api_key=GOOGLE_API_KEY,
    temperature=0.7,
)

embedding = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=GOOGLE_API_KEY)


def ask(prompt: str):
    return llm.invoke([HumanMessage(content=prompt)]).content



In [ ]:
"""
┌─────────────────┐    ┌─────────────┐    ┌──────────────────┐    ┌──────────┐
│  User Question  │ →  │  Retrieve   │ →  │  Generate Answer │ →  │  Answer  │
└─────────────────┘    └─────────────┘    └──────────────────┘    └──────────┘
                         |                                         ↑
                         ↓                                         |
                   ┌─────────────┐                           ┌─────────────┐
                   │  Vector DB  │                           │     LLM     │
                   │ (Qdrant)    │                           │  (Gemini)   │
                   └─────────────┘                           └─────────────┘
                   
"""

In [ ]:
from langchain_community.vectorstores import Qdrant
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_core.documents import Document

# Setup embedding
embedding = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=GOOGLE_API_KEY)

# Inisialisasi Qdrant (in-memory mode)
qdrant_client = QdrantClient(":memory:") # Qdrant bekerja di dalam RAM

# Buat collection dulu secara eksplisit
qdrant_client.recreate_collection(
    collection_name="knowledge_base",
    vectors_config=VectorParams(size=embedding.embed_query("test").__len__(), distance=Distance.COSINE)
)

# Siapkan dokumen
docs = [
    Document(page_content="Python is a programming language."),
    Document(page_content="LangChain is a framework for LLM-powered apps."),
    Document(page_content="Qdrant is a vector database for similarity search."),
]

# Inisialisasi vectorstore
vectorstore = Qdrant(
    client=qdrant_client,
    collection_name="knowledge_base",
    embeddings=embedding
)

# Tambahkan dokumen
vectorstore.add_documents(docs)

C:\Users\Fadhil Wicaksono\AppData\Local\Temp\ipykernel_26716\484950280.py:14: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant_client.recreate_collection(


['456edc39acb64a87a4444d57598e0d0b',
 '26aba15707e045e0825a073a045ad10a',
 '1342973d45294565b697ad7514f834a9']

In [15]:
# Build graph
graph = StateGraph(QAState)
graph.add_node("retrieve", RunnableLambda(retrieve_node))
graph.add_node("generate_answer", RunnableLambda(answer_node))  # ganti nama node
graph.set_entry_point("retrieve")
graph.add_edge("retrieve", "generate_answer")
graph.add_edge("generate_answer", END)

agent = graph.compile()


In [16]:
result = agent.invoke({"question": "Apa itu LangChain?"})
print("Jawaban:\n", result["answer"])

Jawaban:
 LangChain adalah sebuah framework (kerangka kerja) untuk membangun aplikasi yang ditenagai oleh Large Language Models (LLM). Dalam bahasa sederhana, LangChain mempermudah Anda untuk menggunakan model bahasa besar (seperti GPT-3 atau model serupa) untuk membuat berbagai macam aplikasi, seperti chatbot, agen yang bisa menjawab pertanyaan, dan lain sebagainya.

Berikut beberapa poin penting tentang LangChain:

*   **Mempermudah Integrasi LLM:** LangChain menyediakan alat dan abstraksi yang memudahkan Anda untuk menghubungkan dan menggunakan berbagai model LLM.
*   **Rantai (Chains):** Konsep inti LangChain adalah "rantai" (chains), yaitu urutan panggilan LLM atau modul lainnya yang dirangkai untuk mencapai tujuan tertentu.  Ini memungkinkan Anda untuk membuat alur kerja yang kompleks.
*   **Komponen:** LangChain memiliki berbagai komponen, seperti model I/O (untuk berinteraksi dengan LLM), indeks data (untuk mengambil informasi yang relevan), memori (untuk mempertahankan konteks